# Unknown face recognition

In [10]:
# Necessary files for training and inference of model

!cp ./drive/My\ Drive/Mini_casia.zip .
!cp ./drive/My\ Drive/facenet_keras.h5 .
!cp ./drive/My\ Drive/tarantino.mp4 .
!unzip ./Mini_casia

In [ ]:
#Installation of MTCNN and EVM

!pip install mtcnn
#!unzip VGG200.zip
!pip install EVM

In [14]:
%%writefile detect_face_video.py


import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import load_model
import numpy as np
from mtcnn.mtcnn import MTCNN
import os
from numpy import savez_compressed
from numpy import load
from numpy import expand_dims
from numpy import asarray
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC




class detect_face:
    def __init__(self):
        self.detect = MTCNN()
        self.img = None
        self.res = None 
        

    
    def vid_detect_face(self,frame):
        self.img_v = frame
        self.res_v = self.detect.detect_faces(frame)
        return(self.res_v)         
    
    def vid_crop_resize(self):
      self.face = []
      for i in range(len(self.res_v)):
          if self.res_v[i]['confidence']>=0.90:
            x1=self.res_v[i]['box'][0]
            y1=self.res_v[i]['box'][1]
            x2=self.res_v[i]['box'][0]+self.res_v[i]['box'][2]
            y2=self.res_v[i]['box'][1]+self.res_v[i]['box'][3]
            self._face = self.img_v[y1:y2,x1:x2]
            self._face = cv2.resize(self._face,(160,160),cv2.INTER_AREA)
            self.face.append(self._face)
      return(self.face)   
    
    def load_facenet(self):
        self.model = load_model('facenet_keras.h5')
        print('Facenet model loaded')  
    
    def get_embedding(self, face_pixels):

        face_pixels = face_pixels.astype('float32')
        mean, std = face_pixels.mean(), face_pixels.std()
        face_pixels = (face_pixels - mean) / std
        samples = expand_dims(face_pixels, axis=0)
        yhat = self.model.predict(samples)
        return yhat[0]
    
    def normalize(self,data):
        in_encoder = Normalizer(norm='l2')
        data = in_encoder.transform(data)
        return(data)

    def draw_detect(self,img):
    
     l=25
     t=2
     for i in range(len(self.res_v)):
        x1 = self.res_v[i]['box'][0]
        y1 = self.res_v[i]['box'][1]
        x2 = self.res_v[i]['box'][0] + self.res_v[i]['box'][2]
        y2 = self.res_v[i]['box'][1] + self.res_v[i]['box'][3]
        
        cv2.line(img,(x1,y1),(x1+l,y1),(255,0,0),t)
        cv2.line(img,(x1,y1),(x1,y1+l),(255,0,0),t)
        
        cv2.line(img,(x2,y1),(x2-l,y1),(255,0,0),t)
        cv2.line(img,(x2,y1),(x2,y1+l),(255,0,0),t)
        
        cv2.line(img,(x2,y2),(x2-l,y2),(255,0,0),t)
        cv2.line(img,(x2,y2),(x2,y2-l),(255,0,0),t)
       
        cv2.line(img,(x1,y2),(x1+l,y2),(255,0,0),t)
        cv2.line(img,(x1,y2),(x1,y2-l),(255,0,0),t)

        #cv2.line(img,(x1[i]+int(l/2),y1[i]+int(b/outer_len)),(x1[i]+int(l/2),y1[i]-int(b/inner_len)),(0,255,255),6)
        #cv2.line(img,(x1[i]+int(l/outer_len),y1[i]+int(b/2)),(x1[i]-int(l/inner_len),y1[i]+int(b/2)),(0,255,255),6)
        #cv2.line(img,(x1[i]+int(l/2),y1[i]+b-int(b/outer_len)),(x1[i]+int(l/2),y1[i]+b+int(b/inner_len)),(0,255,255),6)
        #cv2.line(img,(x1[i]+int(l)-int(l/outer_len),y1[i]+int(b/2)),(x1[i]+l+int(l/inner_len),y1[i]+int(b/2)),(0,255,255),6)
        
    
     return(img)
        
        


Writing detect_face_video.py


In [15]:
%%writefile extract_embeds.py

#----------------------------------------------------------------------------------------------------
# Train and test folder created are used in this file. The faces are detected using MTCNN
# then these faces are extracted and passed to facenet model to return the embeddings of these faces
#Make sure to load the .h5 facenet file, before running this
#----------------------------------------------------------------------------------------------------

import os
import cv2
import time
from mtcnn.mtcnn import MTCNN
import numpy as np
import tensorflow as tf
import keras
from keras.models import load_model
from numpy import savez_compressed
from numpy import load 
from numpy import expand_dims
from numpy import asarray
from sklearn.preprocessing import Normalizer
import argparse

beginn = time.time()

class extract_embeds:
    def __init__(self,path):
        self.detect = MTCNN()
        self.root_path = path
        self.X_train_faces = []
        self.y_train_faces = []
        self.X_test_faces = []
        self.y_test_faces = []
        self.newTrainX = []
        self.newTestX = []

    def identify_face(self,path):
        self.img = cv2.imread(path)
        self.img = cv2.cvtColor(self.img,cv2.COLOR_BGR2RGB)
        self.res = self.detect.detect_faces(self.img)
        return(self.res)
    
    def crop_resize(self):
        x1=self.res[0]['box'][0]
        y1=self.res[0]['box'][1]
        x2=self.res[0]['box'][0]+self.res[0]['box'][2]
        y2=self.res[0]['box'][1]+self.res[0]['box'][3]
        self.face = self.img[y1:y2,x1:x2]
        np.shape(self.face)
        self.face = cv2.resize(self.face,(160,160),cv2.INTER_AREA)
        return(self.face)
        
    def check_folders(self):
        if 'outputs' not in os.listdir():
            os.makedirs('./outputs')

        y,x = sorted(os.listdir(self.root_path),key=len)
        assert(x == 'train' or x == 'Train' or x == 'TRAIN')
        assert(y == 'test' or y == 'Test' or y == 'TEST')
        self.train_path = self.root_path+'/'+x
        self.test_path = self.root_path+'/'+y
        train_classes = os.listdir(self.train_path)
        test_classes = os.listdir(self.test_path)
        try:
            assert(len(train_classes) == len(test_classes))
            print('No. of train classes: {}'.format(len(train_classes)))
            print('No. of test classes: {}'.format(len(test_classes)))
        except Exception as e:
            print('Error: Number of classes in train and test should be same')
    
    def extract_faces(self):
        start1 = time.time()
        print('Inside train folder ....')
        for label in os.listdir(self.train_path): 
            print('Images of '+label + ' under process')
            for pics in os.listdir(self.train_path+'/'+label):
              if pics.endswith('.jpg'):    
                #print('Name of the pic:',pics)
                try:
                  face_details = self.identify_face(self.train_path+'/'+label+'/'+pics)
                  out = self.crop_resize()
                  self.X_train_faces.append(out)
                  self.y_train_faces.append(label)
                except:
                  print('Pic {} in {} is not processed'.format(pics,label))
                  continue
        print('time taken to process {} secs'.format(time.time()-start1))
    
        start2 = time.time()
        print('Inside test folder ....')
        for label in os.listdir(self.test_path):
            
            print('Images of '+label + ' under process')
            for pics in os.listdir(self.test_path+'/'+label):
              if pics.endswith('.jpg') or pics.endswith('.jpeg') or pics.endswith('.png') or pics.endswith('.tif'):
                #print('Name of the pic:',pics)
                try:
                  face_details = self.identify_face(self.test_path+'/'+label+'/'+pics)
                  out = self.crop_resize()
                  self.X_test_faces.append(out)
                  self.y_test_faces.append(label)
                except:
                  print('Pic {} in {} is not processed'.format(pics,label))
                  continue
        print('time taken to process {} secs'.format(time.time()-start2))
        savez_compressed('./outputs/Extracted_faces.npz', self.X_train_faces, self.y_train_faces, self.X_test_faces, self.y_test_faces)
        return(self.X_train_faces, self.y_train_faces, self.X_test_faces, self.y_test_faces)

    def load_facenet(self):
        self.model = load_model('facenet_keras.h5')
        print('Facenet model loaded')  
    
    def get_embedding(self, face_pixels):

        face_pixels = face_pixels.astype('float32')
        mean, std = face_pixels.mean(), face_pixels.std()
        face_pixels = (face_pixels - mean) / std
        samples = expand_dims(face_pixels, axis=0)
        yhat = self.model.predict(samples)
        return(self.normalize([yhat[0]]))
    
    def normalize(self,data):
        in_encoder = Normalizer(norm='l2')
        data = in_encoder.transform(data)
        return(data)

    def extract_embeddings(self):
        # convert each face in the train set to an embedding
        for face_pixels in self.X_train_faces:
            embedding = self.get_embedding(face_pixels)
            self.newTrainX.append(embedding)
        self.newTrainX = asarray(self.newTrainX)
        print(self.newTrainX.shape)

        # convert each face in the test set to an embedding 
        for face_pixels in self.X_test_faces:
            embedding = self.get_embedding(face_pixels)
            self.newTestX.append(embedding)
        self.newTestX = asarray(self.newTestX)
        print(self.newTestX.shape)                                      
        # save arrays to one file in compressed format
        savez_compressed('./outputs/Extracted_embeddings.npz', self.newTrainX, self.y_train_faces, self.newTestX, self.y_test_faces)
        print('Time taken from reading images to extracting embeds: {} s'.format(time.time()-beginn))
  
if __name__ == '__main__':
     
    parser = argparse.ArgumentParser()
    parser.add_argument('--input_path',default=None, required = True, 
                        help = 'Enter the path of the folder contains train and test images')
    args=parser.parse_args()
    
    _extract_embeds = extract_embeds(args.input_path)
    _extract_embeds.check_folders()
    x,y,a,b = _extract_embeds.extract_faces()
    _extract_embeds.load_facenet()
    _extract_embeds.extract_embeddings()


Writing extract_embeds.py


In [16]:

!python extract_embeds.py --input_path ./Mini_casia


2020-08-17 19:08:37.595550: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-17 19:08:39.549902: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-08-17 19:08:39.609434: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-17 19:08:39.610194: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-08-17 19:08:39.610262: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-17 19:08:39.842224: I tensorflow/stream_executor/platform/default

In [17]:
%%writefile train_evm.py


##------------------------------------------------------------------------------------------------------
# In this code, we are trying to Fit EVM and check the accuracy of fitting all classes
# Any number of classes can be used. It generates, fits each class with label and prints accuracy
# At present this code is used to check only, the known classes
#-------------------------------------------------------------------------------------------------------

import numpy as np
from numpy import load
import EVM, scipy
import time
from sklearn.metrics import accuracy_score
import argparse
import pickle


class evm_acc:

    def __init__(self,embeds_path):

      self.trainX={}
      self.testX = {}
      self.change_test=[]
      self.change_train=[]
      self.predicted_cls_nam = []
      self.embeds_path = embeds_path
      data = load(self.embeds_path)
      #self.tail_size = tail_size
      self.trainx,self.trainy,self.testx,self.testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
    
    
    def class_split(self):

        self.change_test = [i+1 for i in range(len(self.testy)-1) if (self.testy[i] != self.testy[i+1])]
        self.change_train = [i+1 for i in range(len(self.trainy)-1) if (self.trainy[i] != self.trainy[i+1])]

        for i in range(len(self.change_test)):
          if i ==0:
            #self.testX['class_{}'.format(i)] = self.testx[i]
            self.testX['class_{}'.format(i)] = self.testx[i:self.change_test[i]]
          else:
            self.testX['class_{}'.format(i)] = self.testx[self.change_test[i-1]:self.change_test[i]]
          #print(self.testX['class_{}'.format(i)].shape)
        self.testX['class_{}'.format(i+1)] = self.testx[self.change_test[i]:]
        #print(self.testX['class_3'].shape)

        for i in range(len(self.change_train)):
          #print(i)
          if i ==0:
            #self.trainX['class_{}'.format(i)] = self.trainx[i]
            self.trainX['class_{}'.format(i)] = self.trainx[i:self.change_train[i]]
          else:
            self.trainX['class_{}'.format(i)] = self.trainx[self.change_train[i-1]:self.change_train[i]]
          #print(self.trainX['class_{}'.format(i)].shape)
        self.trainX['class_{}'.format(i+1)] = self.trainx[self.change_train[i]:]
        #print(self.trainX['class_3'].shape)

    def fit_evm(self,ts,dm,thresh): 
        print('Fitting the EVM Model .....')
        self.ts = int(ts)
        self.dm = float(dm)
        self.thresh = float(thresh)
        self.mevm = EVM.MultipleEVM(tailsize=self.ts, cover_threshold =self.thresh, distance_multiplier = self.dm, distance_function = scipy.spatial.distance.euclidean)
        self.mevm.train([self.trainX[i] for i in list(self.trainX.keys())])

    def extract_class(self):
      self.indexes = np.unique(self.testy, return_index=True)[1]
      self.uni_classes = [self.testy[index] for index in sorted(self.indexes)]
    
    def test_evm(self):
        for i in range(len(self.testx)):
          self.probs,self.index = self.mevm.max_probabilities([self.testx[i]])
          #print(self.probs,self.index)
          #print(self.probs,self.index,i)
          #print('original',self.testy[i])
          #if self.probs[0] > 0.6:
          self.predicted_cls_nam.append(self.uni_classes[self.index[0][0]])
            #print('class_nam',cls_nam)

    def check_accuracy(self):

        score = accuracy_score(self.testy,self.predicted_cls_nam) 
        print('Distance multiplier {}, cover_threshold {}, tail_size {} and Accuracy {}'.format(self.dm,self.thresh,self.ts,score))
        #for i in range(len(testy)):
          #print(self.testy[i],self.predicted_cls_nam[i])

    def save_model(self):

        with open('./outputs/EVM_model.pkl','wb') as f:
          pickle.dump(self.mevm,f)

        with open('./outputs/class_names.pkl','wb') as t:
            pickle.dump(self.uni_classes,t)

if __name__ == '__main__':

      parser = argparse.ArgumentParser()
      parser.add_argument('--Embeds_path',default=None,required=True,
                          help = "Enter the path of the Extracted embeddings")
      parser.add_argument('--tail_size', default=300,
                          help = "Hyperparameter tail size for EVM model")
      parser.add_argument('--threshold', default=0.7,
                          help = "Hyperparameter threshold value for EVM model")
      parser.add_argument('--dist_multiplier', default=0.7,
                          help = "Hyperparameter distance multiplier for EVM model")
      args = parser.parse_args()


      
      sp = evm_acc(args.Embeds_path)
      sp.class_split()
      sp.fit_evm(args.tail_size,args.threshold,args.dist_multiplier)
      sp.extract_class()
      sp.test_evm() 
      sp.check_accuracy()
      sp.save_model()
      
      



Writing train_evm.py


In [19]:
!python train_evm.py --Embeds_path ./outputs/Extracted_embeddings.npz

Fitting the EVM Model .....
Distance multiplier 0.7, cover_threshold 0.7, tail_size 300 and Accuracy 0.9772727272727273


In [20]:
%%writefile Inference_on_video.py

from detect_face_video import detect_face
import cv2
import pickle
import numpy as np
import argparse


class inference():

  def load_model(self):

      with open('./outputs/EVM_model.pkl','rb') as b:
          self.mevm = pickle.load(b)

      with open('./outputs/class_names.pkl','rb') as cl:
          self.cls_names = pickle.load(cl)

  def read_video(self,path):

      self.cap = cv2.VideoCapture(path)
      width, height = self.cap.get(cv2.CAP_PROP_FRAME_WIDTH), self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
      fourcc=cv2.VideoWriter_fourcc('M','J','P','G')
      #fourcc = cv2.cv.CV_FOURCC(*'XVID')
      self.videowriter = cv2.VideoWriter('./outputs/output_video.avi', fourcc, 20, (int(width),int(height),))

      #Facetection class to be loaded
      self.df = detect_face()
      self.df.load_facenet()

  def run_inference(self):

        while(self.cap.isOpened()):

            # Capture frame-by-frame
            ret, frame0 = self.cap.read()
            if ret == True:
              try:
                  frame = cv2.cvtColor(frame0,cv2.COLOR_BGR2RGB)
                  #frame = cv2.rotate(frame1, cv2.ROTATE_90_COUNTERCLOCKWISE)
                  res = self.df.vid_detect_face(frame)
                  bb_img = self.df.draw_detect(frame)
                  face = self.df.vid_crop_resize()
                  #print(len(res))
                  #print(np.shape(face))
                  for i in range(len(res)):
                    embds = self.df.get_embedding(np.array(face[i]))
                    print(np.shape(embds))
                    x = self.df.normalize([embds])
                    probs,index = self.mevm.max_probabilities(x)
                    #bb_img = bb_img[:,:,::-1]
                    print(probs,self.cls_names[index[0][0]]) 
                    if probs[0] > 0.9:
                        cv2.putText(bb_img,self.cls_names[index[0][0]],(res[i]['box'][0],res[i]['box'][1]-10),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,255),2)
                        self.videowriter.write(bb_img[:,:,::-1])
                    if probs[0] < 0.2:
                        cv2.putText(bb_img,"unknown",(res[i]['box'][0],res[i]['box'][1]-10),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,255),2)
                        self.videowriter.write(bb_img[:,:,::-1])
              except Exception as e:
                  print(e)
                  self.videowriter.write(frame)
                  continue
            else:
              break

        self.videowriter.release()
        self.cap.release()
        cv2.destroyAllWindows()


if __name__ == "__main__":
    print("args.video_path")
    parser = argparse.ArgumentParser()
    parser.add_argument('--video_path',required=True,
                          help = 'Please input the path of the input video')
    args = parser.parse_args()

    inf = inference()
    inf.load_model()
    
    inf.read_video(args.video_path)
    inf.run_inference()

    


Writing Inference_on_video.py


In [21]:
!python Inference_on_video.py --video_path ./tarantino.mp4

2020-08-17 19:24:18.032433: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
args.video_path
2020-08-17 19:24:20.078464: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-08-17 19:24:20.117291: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-17 19:24:20.117944: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-08-17 19:24:20.118011: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-17 19:24:20.119836: I tensorflow/stream_executor/